# import

In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [13]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [14]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [15]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred = logits, y_true = labels,
                                                                   from_logits = True))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [16]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model(class version)

In [17]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(2):
            self.model.add(dense(256,weight_init))
            self.model.add(relu())
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [18]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [19]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [20]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [21]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.8368, train_loss: 2.12459493, train_accuracy: 0.3359, test_accuracy: 0.2305
Epoch: [ 0] [    1/  468] time: 1.0604, train_loss: 2.05547380, train_accuracy: 0.5078, test_accuracy: 0.4472
Epoch: [ 0] [    2/  468] time: 1.2688, train_loss: 1.94714725, train_accuracy: 0.6406, test_accuracy: 0.5841
Epoch: [ 0] [    3/  468] time: 1.4823, train_loss: 1.85712886, train_accuracy: 0.5781, test_accuracy: 0.6336
Epoch: [ 0] [    4/  468] time: 1.6827, train_loss: 1.74581242, train_accuracy: 0.6250, test_accuracy: 0.7068
Epoch: [ 0] [    5/  468] time: 1.8953, train_loss: 1.60289812, train_accuracy: 0.7266, test_accuracy: 0.7454
Epoch: [ 0] [    6/  468] time: 2.1095, train_loss: 1.41714406, train_accuracy: 0.8125, test_accuracy: 0.7532
Epoch: [ 0] [    7/  468] time: 2.3642, train_loss: 1.32083678, train_accuracy: 0.8047, test_accuracy: 0.7452
Epoch: [ 0] [    8/  468] time: 2.5596, 

Epoch: [ 0] [   74/  468] time: 15.1945, train_loss: 0.29702109, train_accuracy: 0.9219, test_accuracy: 0.9172
Epoch: [ 0] [   75/  468] time: 15.3561, train_loss: 0.33096480, train_accuracy: 0.9219, test_accuracy: 0.9137
Epoch: [ 0] [   76/  468] time: 15.5491, train_loss: 0.24061169, train_accuracy: 0.9219, test_accuracy: 0.9122
Epoch: [ 0] [   77/  468] time: 15.7242, train_loss: 0.23351356, train_accuracy: 0.9453, test_accuracy: 0.9112
Epoch: [ 0] [   78/  468] time: 15.8798, train_loss: 0.25935930, train_accuracy: 0.9062, test_accuracy: 0.9153
Epoch: [ 0] [   79/  468] time: 16.0523, train_loss: 0.35509691, train_accuracy: 0.9062, test_accuracy: 0.9186
Epoch: [ 0] [   80/  468] time: 16.2008, train_loss: 0.25127041, train_accuracy: 0.9297, test_accuracy: 0.9194
Epoch: [ 0] [   81/  468] time: 16.3493, train_loss: 0.28900963, train_accuracy: 0.8984, test_accuracy: 0.9191
Epoch: [ 0] [   82/  468] time: 16.4960, train_loss: 0.26177919, train_accuracy: 0.9219, test_accuracy: 0.9176
E

Epoch: [ 0] [  148/  468] time: 28.2566, train_loss: 0.12616660, train_accuracy: 0.9453, test_accuracy: 0.9322
Epoch: [ 0] [  149/  468] time: 28.4265, train_loss: 0.17882405, train_accuracy: 0.9609, test_accuracy: 0.9321
Epoch: [ 0] [  150/  468] time: 28.6027, train_loss: 0.21909916, train_accuracy: 0.9609, test_accuracy: 0.9339
Epoch: [ 0] [  151/  468] time: 28.7838, train_loss: 0.21047217, train_accuracy: 0.9375, test_accuracy: 0.9347
Epoch: [ 0] [  152/  468] time: 28.9683, train_loss: 0.22775900, train_accuracy: 0.9375, test_accuracy: 0.9342
Epoch: [ 0] [  153/  468] time: 29.1542, train_loss: 0.13253279, train_accuracy: 0.9766, test_accuracy: 0.9321
Epoch: [ 0] [  154/  468] time: 29.3354, train_loss: 0.19241250, train_accuracy: 0.9297, test_accuracy: 0.9298
Epoch: [ 0] [  155/  468] time: 29.4953, train_loss: 0.20562752, train_accuracy: 0.9531, test_accuracy: 0.9310
Epoch: [ 0] [  156/  468] time: 29.6812, train_loss: 0.19787806, train_accuracy: 0.9219, test_accuracy: 0.9335
E

Epoch: [ 0] [  223/  468] time: 41.6407, train_loss: 0.25190103, train_accuracy: 0.9453, test_accuracy: 0.9487
Epoch: [ 0] [  224/  468] time: 41.8265, train_loss: 0.10069750, train_accuracy: 0.9688, test_accuracy: 0.9488
Epoch: [ 0] [  225/  468] time: 42.0058, train_loss: 0.14707100, train_accuracy: 0.9375, test_accuracy: 0.9474
Epoch: [ 0] [  226/  468] time: 42.1886, train_loss: 0.15618275, train_accuracy: 0.9609, test_accuracy: 0.9483
Epoch: [ 0] [  227/  468] time: 42.3733, train_loss: 0.18732755, train_accuracy: 0.9297, test_accuracy: 0.9502
Epoch: [ 0] [  228/  468] time: 42.5628, train_loss: 0.16452242, train_accuracy: 0.9453, test_accuracy: 0.9509
Epoch: [ 0] [  229/  468] time: 42.7503, train_loss: 0.18481965, train_accuracy: 0.9609, test_accuracy: 0.9514
Epoch: [ 0] [  230/  468] time: 42.9424, train_loss: 0.13973108, train_accuracy: 0.9688, test_accuracy: 0.9521
Epoch: [ 0] [  231/  468] time: 43.1257, train_loss: 0.09481341, train_accuracy: 0.9688, test_accuracy: 0.9518
E

Epoch: [ 0] [  298/  468] time: 55.2978, train_loss: 0.25830978, train_accuracy: 0.9219, test_accuracy: 0.9555
Epoch: [ 0] [  299/  468] time: 55.4885, train_loss: 0.09514683, train_accuracy: 0.9688, test_accuracy: 0.9558
Epoch: [ 0] [  300/  468] time: 55.6890, train_loss: 0.20419279, train_accuracy: 0.9297, test_accuracy: 0.9534
Epoch: [ 0] [  301/  468] time: 55.8725, train_loss: 0.09034582, train_accuracy: 0.9766, test_accuracy: 0.9515
Epoch: [ 0] [  302/  468] time: 56.0440, train_loss: 0.13581574, train_accuracy: 0.9453, test_accuracy: 0.9514
Epoch: [ 0] [  303/  468] time: 56.2286, train_loss: 0.28262907, train_accuracy: 0.9297, test_accuracy: 0.9516
Epoch: [ 0] [  304/  468] time: 56.3976, train_loss: 0.16372031, train_accuracy: 0.9609, test_accuracy: 0.9525
Epoch: [ 0] [  305/  468] time: 56.5900, train_loss: 0.13508983, train_accuracy: 0.9688, test_accuracy: 0.9541
Epoch: [ 0] [  306/  468] time: 56.7676, train_loss: 0.11497711, train_accuracy: 0.9688, test_accuracy: 0.9551
E

Epoch: [ 0] [  373/  468] time: 68.4088, train_loss: 0.20004717, train_accuracy: 0.9297, test_accuracy: 0.9523
Epoch: [ 0] [  374/  468] time: 68.5989, train_loss: 0.19687952, train_accuracy: 0.9375, test_accuracy: 0.9528
Epoch: [ 0] [  375/  468] time: 68.7545, train_loss: 0.15075421, train_accuracy: 0.9531, test_accuracy: 0.9547
Epoch: [ 0] [  376/  468] time: 68.9340, train_loss: 0.07597910, train_accuracy: 0.9844, test_accuracy: 0.9545
Epoch: [ 0] [  377/  468] time: 69.0917, train_loss: 0.17093554, train_accuracy: 0.9453, test_accuracy: 0.9534
Epoch: [ 0] [  378/  468] time: 69.2396, train_loss: 0.25350618, train_accuracy: 0.9453, test_accuracy: 0.9552
Epoch: [ 0] [  379/  468] time: 69.3981, train_loss: 0.15031904, train_accuracy: 0.9531, test_accuracy: 0.9568
Epoch: [ 0] [  380/  468] time: 69.5487, train_loss: 0.10161178, train_accuracy: 0.9766, test_accuracy: 0.9584
Epoch: [ 0] [  381/  468] time: 69.7162, train_loss: 0.08589731, train_accuracy: 0.9922, test_accuracy: 0.9605
E

Epoch: [ 0] [  448/  468] time: 81.4629, train_loss: 0.10077465, train_accuracy: 0.9688, test_accuracy: 0.9647
Epoch: [ 0] [  449/  468] time: 81.6403, train_loss: 0.14260855, train_accuracy: 0.9531, test_accuracy: 0.9641
Epoch: [ 0] [  450/  468] time: 81.7982, train_loss: 0.15642066, train_accuracy: 0.9609, test_accuracy: 0.9636
Epoch: [ 0] [  451/  468] time: 81.9627, train_loss: 0.17154980, train_accuracy: 0.9453, test_accuracy: 0.9624
Epoch: [ 0] [  452/  468] time: 82.1425, train_loss: 0.13599816, train_accuracy: 0.9609, test_accuracy: 0.9614
Epoch: [ 0] [  453/  468] time: 82.3230, train_loss: 0.10690701, train_accuracy: 0.9609, test_accuracy: 0.9602
Epoch: [ 0] [  454/  468] time: 82.5115, train_loss: 0.17139545, train_accuracy: 0.9688, test_accuracy: 0.9610
Epoch: [ 0] [  455/  468] time: 82.6980, train_loss: 0.12227364, train_accuracy: 0.9609, test_accuracy: 0.9623
Epoch: [ 0] [  456/  468] time: 82.8825, train_loss: 0.06211453, train_accuracy: 0.9844, test_accuracy: 0.9635
E

In [22]:
# test acc = 95.99